In [1]:
import os
import tensorflow as tf
from PIL import UnidentifiedImageError
from tensorflow.keras.preprocessing.image import (load_img, img_to_array, ImageDataGenerator)
from tensorflow.keras.applications.vgg16 import (preprocess_input, decode_predictions, VGG16)
from tensorflow.keras.layers import (Flatten, Dense, Dropout, BatchNormalization)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import SGD, Adam
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from scikeras.wrappers import KerasClassifier

2024-05-16 09:29:05.533280: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-16 09:29:05.537103: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-16 09:29:05.588763: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-16 09:29:06.443311: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
from gridsearch import gridsearch

In [2]:

def parser():
    """
    The user can specify whether to perform GridSearch, number of epochs, batch size, and data augmentation.
    The function will then parse command-line arguments and make them lower case.
    """
    parser = argparse.ArgumentParser()
    parser.add_argument("--GridSearch",
                        "-gs",
                        required = True,
                        choices = ["yes", "no"],
                        help = "Perform GridSearch (yes or no)")
    parser.add_argument("--BatchNorm",
                        "-bn",
                        required = True,
                        choices = ["yes", "no"],
                        help = "Perform batch normalization (yes or no)")   
    parser.add_argument("--DatAug",
                        "-da",
                        required = True,
                        choices = ["yes", "no"],
                        help = "Perform data augmentation (yes or no)")                
    args = parser.parse_args()
    args.GridSearch = args.GridSearch.lower()
    args.BatchNorm = args.BatchNorm.lower()
    args.DatAug = args.DatAug.lower()
    return args



def load_images(folder_path):
    """
    Loads the data from the specified folder path, generates labels for each image, 
    and preprocesses them for model input.
    The dataset contains certain files, i.e., Thumbs.db, could not be loaded and
    returned the error 'UnidentifiedImageError'. These will simplt be ignored.
    """
    list_of_images = [] 
    list_of_labels = []
    
    for subfolder in sorted(os.listdir(folder_path)):
        subfolder_path  = os.path.join(folder_path, subfolder)
        
        for file in os.listdir(subfolder_path):
            individual_filepath = os.path.join(subfolder_path, file)
            
            try:
                image = load_img(individual_filepath, target_size = (224, 224))
                image = img_to_array(image)
                list_of_images.append(image)

                label = subfolder_path.split("/")[-1]
                list_of_labels.append(label)

            except (UnidentifiedImageError):
                print(f"Skipping {individual_filepath}")
        
    array_of_images = np.array(list_of_images)
    X = preprocess_input(array_of_images)
    y = list_of_labels
    
    return X, y


def data_split(X, y):
    """
    Splits the data into training and testing sets by stratifing y.
    Normalizes X to range between 0 and 1 and performs label binarization on y.
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 123)
    X_train = X_train.astype("float32") / 255.
    X_test = X_test.astype("float32") / 255.
    lb = LabelBinarizer()
    y_train = lb.fit_transform(y_train)
    y_test = lb.fit_transform(y_test) 

    return X_train, X_test, y_train, y_test



def define_model(BatchNorm):
    """
    Defines the model architecture. First, the VGG16 model is loaded from TensorFlow without the classification
    layers and the convolutional layers are marked as not trainable to retain their pretrained weights.
    Subsequently, a new fully connected layer with ReLU activation is added followed by an output layer with
    softmax activation for multi-class classification.

    batchNorm option --> user specifies whether the model should be defined with or without batch normalization

    compile

    """
    model = VGG16(include_top = False, pooling = 'avg', input_shape = (224, 224, 3))

    for layer in model.layers:
        layer.trainable = False

    if BatchNorm == "no":
        flat1 = Flatten()(model.layers[-1].output)
        class1 = Dense(128, activation = 'relu')(flat1)
        output = Dense(10, activation = 'softmax')(class1)

    if BatchNorm == "yes":
        flat1 = Flatten()(model.layers[-1].output)
        bn = BatchNormalization()(flat1)
        class1 = Dense(128, activation='relu')(bn)
        output = Dense(10, activation='softmax')(class1)
    
    model = Model(inputs = model.inputs, outputs = output)

    # compile
    #model.compile(loss = "categorical_crossentropy", metrics = ["accuracy"],
    #            optimizer = optimizer, learning_rate = learning_rate,
    #            epochs = epochs, batch_size = batch_size)

    return model


def sklearn_object(model):
    """
    Convert the model from a KerasClassifier to an object, that can be used in a scikit-learn pipeline.
    """
    model = KerasClassifier(model = model, verbose = 0)
    return model


def data_generator():
    """
    Creates an image data generator with data augmentation settings as horizontal flipping and rotation.
    """
    datagen = ImageDataGenerator(horizontal_flip = True, 
                                rotation_range = 90,
                                validation_split = 0.1)
    return datagen


def compile_fit_model(model, X_train, y_train, model_config_default, DatAug):
    """
    Compiles the model with the specified optimizer.

    The user specifies whether to implement data augmentation...
    Fits the compiled model to the training data with or without data augmentation and returns the training history.
    """
    model.compile(loss = "categorical_crossentropy", metrics = ["accuracy"],
                optimizer = optimizer, learning_rate = learning_rate,
                epochs = epochs, batch_size = batch_size)
    model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'].
                optimizer = model_config_default.optimizer, learning_rate = model_config_default.learning_rate,
                epochs = model_config_default.epochs, batch_size = model_config_default.batch_size)

    if DatAug == "no":
        H = model.fit(X_train, y_train, 
                    validation_split = 0.1,
                    model_config_default,
                    batch_size = model_config_default.batch_size, # 32
                    epochs = model_config_default.epochs, #10
                    verbose = 1)
    elif DatAug == "yes":
        DatAug = data_generator()
        datagen.fit(X_train)
        H = model.fit(datagen.flow(X_train, y_train, batch_size = BatchSize),
                                    validation_data = datagen.flow(X_train, y_train, 
                                                                    batch_size = BatchSize,
                                                                    subset = "validation"),
                                                                    epochs = epochs) 
    return H



def plot_history(H, epochs, outpath):
    """
    Plots the training and validation loss and accuracy curves and saves the plot.
    """
    plt.figure(figsize = (12,6))
    plt.subplot(1,2,1)
    plt.plot(np.arange(0, epochs), H.history["loss"], label = "train_loss")
    plt.plot(np.arange(0, epochs), H.history["val_loss"], label = "val_loss", linestyle = ":")
    plt.title("Loss curve")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.tight_layout()
    plt.legend()
    plt.subplot(1,2,2)
    plt.plot(np.arange(0, epochs), H.history["accuracy"], label = "train_acc")
    plt.plot(np.arange(0, epochs), H.history["val_accuracy"], label = "val_acc", linestyle = ":")
    plt.title("Accuracy curve")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.tight_layout()
    plt.legend()
    plt.show()
    plt.savefig(outpath)


def evaluate(X_test, y_test, model, H, BatchSize, epochs):
    """
    Evaluates the model on the test data, generates classification reports, and saves the results.
    """
    label_names = ["ADVE", "Email", "Form", "Letter", "Memo", "News", "Note", "Report", "Resume", "Scientific"]

    predictions = model.predict(X_test, batch_size = BatchSize)

    classifier_metrics = classification_report(y_test.argmax(axis = 1),
                                               predictions.argmax(axis = 1),
                                               target_names = label_names)

    filepath_metrics = open('out/VGG16_metrics_BatchNorm.txt', 'w')
    filepath_metrics.write(classifier_metrics)
    filepath_metrics.close()

    plot_history(H, epochs, "out/VGG16_losscurve_BatchNorm.png")

    return print("Results have been saved to the out folder")




In [4]:
class ModelConfig_default:
    
    def __init__(self):
        self.learning_rate = self.learning_rate_schedule()
        self.epochs = 10
        self.batch_size = 32

    def learning_rate_schedule(self):
        return tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate = 0.001,
            decay_steps = 10000,
            decay_rate = 0.9)

In [ ]:

# define model
# - gridsearch on defined model? then it can be compiled with the best hyperparameters
# compile model
# - gridsearch on compiled model?
# fit model

# classes

# add compile to define, ligesom week 12

def main():
    
    args = parser()

    folder_path = os.path.join("../../../../cds-vis-data/Tobacco3482") # ("in/Tobacco3482")

    X, y = load_images(folder_path)
    X_train, X_test, y_train, y_test = data_split(X, y)
    
    # define model - BatchNorm yes/no
    define_model(args.BatchNorm)

    # grid search
    if args.GridSearch == 'yes':
        model = sklearn_object(model)
        model = gridsearch(model, X_train, y_train)
    else:
        # load the default model
        model = ModelConfig_default() # default (adam, default lr 0.001, 10 epochs, batch 32) 

    # compile with GridSearch parameters or default + fit model
    H = compile_fit_model(model, X_train, y_train, model_config_default, args.DatAug)
    # evaluate
    evaluate(X_test, y_test, model, H)

    # plotter 

if __name__ == "__main__":
    main()

In [ ]:
folder_path = os.path.join("../../../../../cds-vis-data/Tobacco3482") # ("in/Tobacco3482")

X, y = load_images(folder_path)
X_train, X_test, y_train, y_test = data_split(X, y)

In [10]:
define_model('yes')

<Functional name=functional_3, built=True>